# Machine Learning Engineer Nanodegree

## Capstone Project - Smile Detector


---

In this project, CNNs are used to build models to detect if the person in the image is smiling or not.
CelibA dataset is used for this purpose - http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html
The data for this project is downloaded from Kaggle - https://www.kaggle.com/jessicali9530/celeba-dataset
Here, transfer learning with a pretrained model VGG19 is used with random weights and SGD optimizer.

In [15]:
IMG_H=218
IMG_W=178
IMG_D=3

In the code cell below, we populate a few variables through the use of the load_files function from the scikit-learn library:

train_files, valid_files, test_files - numpy arrays containing file paths to images train_targets, valid_targets, test_targets - numpy arrays containing onehot-encoded classification labels smile_names - list of string-valued smile categories for translating labels.

In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path, load_content=False)
    smile_files = np.array(data['filenames'])
    smile_targets = np_utils.to_categorical(np.array(data['target']), 2)
    return smile_files, smile_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('input/dataset/train')
valid_files, valid_targets = load_dataset('input/dataset/validate')
test_files, test_targets = load_dataset('input/dataset/test')

smile_names = [item[:-1] for item in sorted(glob("input/dataset/train/*/"))]

# print statistics about the dataset
print('There are %d total smile categories.' % len(smile_names))
print('There are %s total smile images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training smile images.' % len(train_files))
print('There are %d validation smile images.' % len(valid_files))
print('There are %d test smile images.'% len(test_files))

Using TensorFlow backend.


There are 2 total smile categories.
There are 15000 total smile images.

There are 10000 training smile images.
There are 2500 validation smile images.
There are 2500 test smile images.


When using TensorFlow as backend, Keras CNNs require a 4D array as input. Below is the function for the same. Also, preprocess input is used from VGG19 which is the chosen pre-trained model

In [3]:
from tqdm import tqdm
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(IMG_H, IMG_W))
    # convert PIL.Image.Image type to 3D tensor with shape (IMG_H, IMG_W, IMG_D)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, IMG_H, IMG_W, IMG_D) and return 4D tensor
    x = preprocess_input(x)
    return np.expand_dims(x, axis=0)
    
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files)
valid_tensors = paths_to_tensor(valid_files)
test_tensors = paths_to_tensor(test_files)

100%|██████████| 2500/2500 [01:17<00:00, 32.08it/s]


### Understanding the VGG19 model.

In [5]:
orig_model = VGG19(weights='imagenet',
                  include_top=False,
                    input_shape=(IMG_H, IMG_W, IMG_D))
print("number of layers:", len(orig_model.layers))
orig_model.summary()

number of layers: 22
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 218, 178, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 218, 178, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 218, 178, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 109, 89, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 109, 89, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 109, 89, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 54, 


After compiling the below mentioned model with rmsprop optimizer, trying to explore with SGD (Stochastic Gradient Descent). Started with a very low learning rate of 0.0001 and momentum 0.9

In [6]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.optimizers import SGD

newLayer = orig_model.output
newLayer = GlobalAveragePooling2D()(newLayer)
newLayer = Dense(1024, activation='relu')(newLayer)
newLayer = Dropout(0.25)(newLayer)
newLayer = Dense(512, activation="relu")(newLayer)
newLayer = Dropout(0.25)(newLayer);
predictions = Dense(2,activation='softmax')(newLayer)
newModel = Model(inputs=orig_model.input,outputs=predictions)

# train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in newModel.layers[:22]:
    layer.trainable = False


In [7]:
# compile the model (should be done *after* setting layers to non-trainable)
newModel.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
newModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 218, 178, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 218, 178, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 218, 178, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 109, 89, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 109, 89, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 109, 89, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 54, 44, 128)       0   

In [9]:
trainable = 0
for layer in newModel.layers:
    if (layer.trainable == True):
            trainable = trainable + 1;
print ("trainable Layers = ", trainable)            

trainable Layers =  6


In [10]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='input/saved_models/transfer_models_SGD.weights.best.hdf5',
                               verbose=1,save_best_only=True)

#### Fit the model

In [11]:
NUM_EPOCHS=20
BATCH_SIZE=32
newModel.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpointer], verbose=1)

Train on 10000 samples, validate on 2500 samples
Epoch 1/20


 5600/10000 [===============>..............] - ETA: 1:41:48 - loss: 1.2278 - accuracy: 0.500 - ETA: 1:24:33 - loss: 1.4656 - accuracy: 0.437 - ETA: 1:19:30 - loss: 1.4503 - accuracy: 0.447 - ETA: 1:16:40 - loss: 1.5574 - accuracy: 0.445 - ETA: 1:14:31 - loss: 1.4226 - accuracy: 0.487 - ETA: 1:13:06 - loss: 1.5547 - accuracy: 0.453 - ETA: 1:12:04 - loss: 1.5212 - accuracy: 0.446 - ETA: 1:11:19 - loss: 1.5544 - accuracy: 0.460 - ETA: 1:10:31 - loss: 1.4991 - accuracy: 0.472 - ETA: 1:09:54 - loss: 1.5241 - accuracy: 0.456 - ETA: 1:09:17 - loss: 1.4949 - accuracy: 0.465 - ETA: 1:08:42 - loss: 1.4903 - accuracy: 0.460 - ETA: 1:08:01 - loss: 1.4768 - accuracy: 0.461 - ETA: 1:07:28 - loss: 1.4439 - accuracy: 0.473 - ETA: 1:07:05 - loss: 1.4548 - accuracy: 0.470 - ETA: 1:06:39 - loss: 1.4456 - accuracy: 0.476 - ETA: 1:06:13 - loss: 1.4215 - accuracy: 0.477 - ETA: 1:05:43 - loss: 1.4226 - accuracy: 0.475 - ETA: 1:05:20 - loss: 1.4047 - accuracy: 0.480 - ETA: 1:05:02 - loss: 1.4002 - accuracy: 0

10000/10000 [==============================] - ETA: 29:44 - loss: 0.9615 - accuracy: 0.606 - ETA: 29:30 - loss: 0.9619 - accuracy: 0.606 - ETA: 29:17 - loss: 0.9605 - accuracy: 0.606 - ETA: 29:04 - loss: 0.9600 - accuracy: 0.606 - ETA: 28:51 - loss: 0.9589 - accuracy: 0.607 - ETA: 28:37 - loss: 0.9587 - accuracy: 0.607 - ETA: 28:25 - loss: 0.9585 - accuracy: 0.607 - ETA: 28:12 - loss: 0.9586 - accuracy: 0.607 - ETA: 27:59 - loss: 0.9584 - accuracy: 0.607 - ETA: 27:45 - loss: 0.9564 - accuracy: 0.608 - ETA: 27:32 - loss: 0.9553 - accuracy: 0.608 - ETA: 27:19 - loss: 0.9525 - accuracy: 0.609 - ETA: 27:06 - loss: 0.9523 - accuracy: 0.610 - ETA: 26:53 - loss: 0.9525 - accuracy: 0.610 - ETA: 26:40 - loss: 0.9513 - accuracy: 0.610 - ETA: 26:27 - loss: 0.9519 - accuracy: 0.609 - ETA: 26:14 - loss: 0.9498 - accuracy: 0.610 - ETA: 26:02 - loss: 0.9495 - accuracy: 0.610 - ETA: 25:48 - loss: 0.9490 - accuracy: 0.610 - ETA: 25:35 - loss: 0.9475 - accuracy: 0.610 - ETA: 25:22 - loss: 0.9465 - accur

 5632/10000 [===============>..............] - ETA: 1:08:58 - loss: 0.5119 - accuracy: 0.750 - ETA: 1:07:54 - loss: 0.6015 - accuracy: 0.703 - ETA: 1:06:45 - loss: 0.5699 - accuracy: 0.708 - ETA: 1:06:24 - loss: 0.6122 - accuracy: 0.703 - ETA: 1:05:53 - loss: 0.6325 - accuracy: 0.687 - ETA: 1:05:46 - loss: 0.6581 - accuracy: 0.682 - ETA: 1:05:32 - loss: 0.6509 - accuracy: 0.683 - ETA: 1:05:00 - loss: 0.6373 - accuracy: 0.687 - ETA: 1:04:45 - loss: 0.6289 - accuracy: 0.684 - ETA: 1:04:32 - loss: 0.6660 - accuracy: 0.671 - ETA: 1:04:39 - loss: 0.6645 - accuracy: 0.670 - ETA: 1:04:24 - loss: 0.6475 - accuracy: 0.677 - ETA: 1:04:07 - loss: 0.6711 - accuracy: 0.668 - ETA: 1:03:54 - loss: 0.6869 - accuracy: 0.665 - ETA: 1:03:40 - loss: 0.7214 - accuracy: 0.654 - ETA: 1:03:22 - loss: 0.7141 - accuracy: 0.662 - ETA: 1:03:07 - loss: 0.7097 - accuracy: 0.663 - ETA: 1:02:53 - loss: 0.7167 - accuracy: 0.659 - ETA: 1:02:38 - loss: 0.7127 - accuracy: 0.666 - ETA: 1:02:25 - loss: 0.7063 - accuracy: 0

10000/10000 [==============================] - ETA: 29:05 - loss: 0.6850 - accuracy: 0.672 - ETA: 28:53 - loss: 0.6849 - accuracy: 0.672 - ETA: 28:40 - loss: 0.6852 - accuracy: 0.672 - ETA: 28:27 - loss: 0.6841 - accuracy: 0.672 - ETA: 28:14 - loss: 0.6828 - accuracy: 0.673 - ETA: 28:01 - loss: 0.6819 - accuracy: 0.673 - ETA: 27:49 - loss: 0.6810 - accuracy: 0.673 - ETA: 27:36 - loss: 0.6810 - accuracy: 0.673 - ETA: 27:23 - loss: 0.6810 - accuracy: 0.673 - ETA: 27:11 - loss: 0.6799 - accuracy: 0.674 - ETA: 26:58 - loss: 0.6805 - accuracy: 0.674 - ETA: 26:45 - loss: 0.6804 - accuracy: 0.674 - ETA: 26:33 - loss: 0.6791 - accuracy: 0.674 - ETA: 26:20 - loss: 0.6782 - accuracy: 0.675 - ETA: 26:07 - loss: 0.6790 - accuracy: 0.675 - ETA: 25:54 - loss: 0.6799 - accuracy: 0.674 - ETA: 25:42 - loss: 0.6808 - accuracy: 0.673 - ETA: 25:29 - loss: 0.6818 - accuracy: 0.672 - ETA: 25:16 - loss: 0.6805 - accuracy: 0.673 - ETA: 25:03 - loss: 0.6812 - accuracy: 0.673 - ETA: 24:51 - loss: 0.6815 - accur

...

10000/10000 [==============================] - ETA: 30:24 - loss: 0.4958 - accuracy: 0.755 - ETA: 30:11 - loss: 0.4957 - accuracy: 0.755 - ETA: 29:57 - loss: 0.4950 - accuracy: 0.756 - ETA: 29:44 - loss: 0.4954 - accuracy: 0.755 - ETA: 29:31 - loss: 0.4948 - accuracy: 0.756 - ETA: 29:17 - loss: 0.4953 - accuracy: 0.756 - ETA: 29:03 - loss: 0.4950 - accuracy: 0.756 - ETA: 28:49 - loss: 0.4946 - accuracy: 0.757 - ETA: 28:37 - loss: 0.4945 - accuracy: 0.757 - ETA: 28:23 - loss: 0.4939 - accuracy: 0.757 - ETA: 28:11 - loss: 0.4936 - accuracy: 0.757 - ETA: 27:58 - loss: 0.4940 - accuracy: 0.757 - ETA: 27:45 - loss: 0.4935 - accuracy: 0.757 - ETA: 27:32 - loss: 0.4937 - accuracy: 0.757 - ETA: 27:18 - loss: 0.4935 - accuracy: 0.757 - ETA: 27:04 - loss: 0.4940 - accuracy: 0.757 - ETA: 26:51 - loss: 0.4935 - accuracy: 0.757 - ETA: 26:37 - loss: 0.4934 - accuracy: 0.757 - ETA: 26:23 - loss: 0.4933 - accuracy: 0.757 - ETA: 26:09 - loss: 0.4926 - accuracy: 0.757 - ETA: 25:55 - loss: 0.4929 - accur

 5632/10000 [===============>..............] - ETA: 1:08:07 - loss: 0.5380 - accuracy: 0.718 - ETA: 1:07:42 - loss: 0.5501 - accuracy: 0.781 - ETA: 1:06:56 - loss: 0.5460 - accuracy: 0.750 - ETA: 1:07:26 - loss: 0.5865 - accuracy: 0.718 - ETA: 1:07:33 - loss: 0.5889 - accuracy: 0.700 - ETA: 1:07:35 - loss: 0.5624 - accuracy: 0.724 - ETA: 1:07:31 - loss: 0.5692 - accuracy: 0.732 - ETA: 1:07:26 - loss: 0.5602 - accuracy: 0.750 - ETA: 1:07:58 - loss: 0.5422 - accuracy: 0.763 - ETA: 1:07:53 - loss: 0.5330 - accuracy: 0.768 - ETA: 1:07:51 - loss: 0.5425 - accuracy: 0.761 - ETA: 1:08:45 - loss: 0.5524 - accuracy: 0.752 - ETA: 1:09:37 - loss: 0.5489 - accuracy: 0.745 - ETA: 1:09:37 - loss: 0.5427 - accuracy: 0.752 - ETA: 1:09:17 - loss: 0.5295 - accuracy: 0.760 - ETA: 1:09:08 - loss: 0.5168 - accuracy: 0.767 - ETA: 1:09:26 - loss: 0.5195 - accuracy: 0.761 - ETA: 1:09:40 - loss: 0.5155 - accuracy: 0.760 - ETA: 1:09:14 - loss: 0.5110 - accuracy: 0.763 - ETA: 1:08:41 - loss: 0.5063 - accuracy: 0

10000/10000 [==============================] - ETA: 29:19 - loss: 0.4885 - accuracy: 0.764 - ETA: 29:06 - loss: 0.4888 - accuracy: 0.764 - ETA: 28:53 - loss: 0.4881 - accuracy: 0.765 - ETA: 28:39 - loss: 0.4878 - accuracy: 0.765 - ETA: 28:26 - loss: 0.4879 - accuracy: 0.764 - ETA: 28:13 - loss: 0.4875 - accuracy: 0.765 - ETA: 28:01 - loss: 0.4890 - accuracy: 0.765 - ETA: 27:48 - loss: 0.4897 - accuracy: 0.764 - ETA: 27:34 - loss: 0.4899 - accuracy: 0.763 - ETA: 27:21 - loss: 0.4911 - accuracy: 0.762 - ETA: 27:08 - loss: 0.4909 - accuracy: 0.763 - ETA: 26:55 - loss: 0.4904 - accuracy: 0.763 - ETA: 26:42 - loss: 0.4898 - accuracy: 0.763 - ETA: 26:29 - loss: 0.4896 - accuracy: 0.764 - ETA: 26:16 - loss: 0.4894 - accuracy: 0.764 - ETA: 26:03 - loss: 0.4904 - accuracy: 0.763 - ETA: 25:50 - loss: 0.4911 - accuracy: 0.762 - ETA: 25:37 - loss: 0.4912 - accuracy: 0.762 - ETA: 25:24 - loss: 0.4907 - accuracy: 0.763 - ETA: 25:10 - loss: 0.4904 - accuracy: 0.762 - ETA: 24:57 - loss: 0.4903 - accur

 5632/10000 [===============>..............] - ETA: 1:06:14 - loss: 0.4527 - accuracy: 0.781 - ETA: 1:06:11 - loss: 0.4670 - accuracy: 0.781 - ETA: 1:05:51 - loss: 0.5142 - accuracy: 0.781 - ETA: 1:05:19 - loss: 0.5016 - accuracy: 0.804 - ETA: 1:05:19 - loss: 0.5010 - accuracy: 0.787 - ETA: 1:05:22 - loss: 0.5194 - accuracy: 0.770 - ETA: 1:05:23 - loss: 0.5066 - accuracy: 0.772 - ETA: 1:05:17 - loss: 0.5157 - accuracy: 0.765 - ETA: 1:04:54 - loss: 0.4981 - accuracy: 0.774 - ETA: 1:04:37 - loss: 0.4999 - accuracy: 0.768 - ETA: 1:04:22 - loss: 0.5152 - accuracy: 0.750 - ETA: 1:04:10 - loss: 0.5263 - accuracy: 0.739 - ETA: 1:03:56 - loss: 0.5200 - accuracy: 0.742 - ETA: 1:03:42 - loss: 0.5111 - accuracy: 0.750 - ETA: 1:03:29 - loss: 0.5106 - accuracy: 0.747 - ETA: 1:03:17 - loss: 0.5119 - accuracy: 0.742 - ETA: 1:03:04 - loss: 0.5089 - accuracy: 0.744 - ETA: 1:02:53 - loss: 0.5023 - accuracy: 0.746 - ETA: 1:02:37 - loss: 0.5004 - accuracy: 0.751 - ETA: 1:02:23 - loss: 0.4961 - accuracy: 0

10000/10000 [==============================] - ETA: 29:12 - loss: 0.4916 - accuracy: 0.762 - ETA: 28:59 - loss: 0.4917 - accuracy: 0.762 - ETA: 28:47 - loss: 0.4923 - accuracy: 0.762 - ETA: 28:34 - loss: 0.4916 - accuracy: 0.763 - ETA: 28:21 - loss: 0.4909 - accuracy: 0.763 - ETA: 28:08 - loss: 0.4913 - accuracy: 0.763 - ETA: 27:56 - loss: 0.4907 - accuracy: 0.764 - ETA: 27:43 - loss: 0.4900 - accuracy: 0.764 - ETA: 27:31 - loss: 0.4904 - accuracy: 0.764 - ETA: 27:18 - loss: 0.4911 - accuracy: 0.763 - ETA: 27:06 - loss: 0.4906 - accuracy: 0.764 - ETA: 26:53 - loss: 0.4906 - accuracy: 0.764 - ETA: 26:40 - loss: 0.4909 - accuracy: 0.763 - ETA: 26:27 - loss: 0.4904 - accuracy: 0.764 - ETA: 26:14 - loss: 0.4898 - accuracy: 0.764 - ETA: 26:02 - loss: 0.4897 - accuracy: 0.764 - ETA: 25:49 - loss: 0.4892 - accuracy: 0.764 - ETA: 25:36 - loss: 0.4889 - accuracy: 0.765 - ETA: 25:23 - loss: 0.4891 - accuracy: 0.764 - ETA: 25:10 - loss: 0.4895 - accuracy: 0.764 - ETA: 24:58 - loss: 0.4892 - accur

In [12]:
# Load the model with the best validation accuracy
newModel.load_weights('input/saved_models/transfer_models_SGD.weights.best.hdf5')

In [13]:
loop=0

match=0
nonmatch=0

for test_data in test_tensors:
    pred = newModel.predict(np.expand_dims(test_data, axis=0))
    prediction = np.argmax(pred)
    if (np.argmax(test_targets[loop], axis=0) == prediction):
        match = match + 1
    else:
        nonmatch = nonmatch+1
    loop = loop + 1
    print ("I = {} M = {} D = {} ".format (loop, match, nonmatch))


I = 1 M = 1 D = 0 
I = 2 M = 1 D = 1 
I = 3 M = 2 D = 1 
I = 4 M = 3 D = 1 
I = 5 M = 4 D = 1 
I = 6 M = 5 D = 1 
I = 7 M = 5 D = 2 
I = 8 M = 6 D = 2 
I = 9 M = 7 D = 2 
I = 10 M = 8 D = 2 
I = 11 M = 9 D = 2 
I = 12 M = 10 D = 2 
I = 13 M = 11 D = 2 
I = 14 M = 11 D = 3 
I = 15 M = 12 D = 3 
I = 16 M = 13 D = 3 
I = 17 M = 13 D = 4 
I = 18 M = 13 D = 5 
I = 19 M = 14 D = 5 
I = 20 M = 14 D = 6 
I = 21 M = 15 D = 6 
I = 22 M = 16 D = 6 
I = 23 M = 17 D = 6 
I = 24 M = 17 D = 7 
I = 25 M = 17 D = 8 
I = 26 M = 18 D = 8 
I = 27 M = 19 D = 8 
I = 28 M = 20 D = 8 
I = 29 M = 21 D = 8 
I = 30 M = 22 D = 8 
I = 31 M = 22 D = 9 
I = 32 M = 23 D = 9 
I = 33 M = 23 D = 10 
I = 34 M = 24 D = 10 
I = 35 M = 24 D = 11 
I = 36 M = 24 D = 12 
I = 37 M = 25 D = 12 
I = 38 M = 26 D = 12 
I = 39 M = 27 D = 12 
I = 40 M = 28 D = 12 
I = 41 M = 29 D = 12 
I = 42 M = 29 D = 13 
I = 43 M = 29 D = 14 
I = 44 M = 29 D = 15 
I = 45 M = 29 D = 16 
I = 46 M = 30 D = 16 
I = 47 M = 31 D = 16 
I = 48 M = 32 D = 

I = 353 M = 240 D = 113 
I = 354 M = 241 D = 113 
I = 355 M = 242 D = 113 
I = 356 M = 243 D = 113 
I = 357 M = 244 D = 113 
I = 358 M = 245 D = 113 
I = 359 M = 246 D = 113 
I = 360 M = 247 D = 113 
I = 361 M = 247 D = 114 
I = 362 M = 248 D = 114 
I = 363 M = 248 D = 115 
I = 364 M = 249 D = 115 
I = 365 M = 249 D = 116 
I = 366 M = 250 D = 116 
I = 367 M = 251 D = 116 
I = 368 M = 251 D = 117 
I = 369 M = 252 D = 117 
I = 370 M = 253 D = 117 
I = 371 M = 253 D = 118 
I = 372 M = 254 D = 118 
I = 373 M = 255 D = 118 
I = 374 M = 256 D = 118 
I = 375 M = 257 D = 118 
I = 376 M = 258 D = 118 
I = 377 M = 258 D = 119 
I = 378 M = 259 D = 119 
I = 379 M = 260 D = 119 
I = 380 M = 261 D = 119 
I = 381 M = 262 D = 119 
I = 382 M = 262 D = 120 
I = 383 M = 262 D = 121 
I = 384 M = 263 D = 121 
I = 385 M = 263 D = 122 
I = 386 M = 264 D = 122 
I = 387 M = 265 D = 122 
I = 388 M = 266 D = 122 
I = 389 M = 267 D = 122 
I = 390 M = 268 D = 122 
I = 391 M = 269 D = 122 
I = 392 M = 270 D = 122 


I = 680 M = 478 D = 202 
I = 681 M = 478 D = 203 
I = 682 M = 479 D = 203 
I = 683 M = 480 D = 203 
I = 684 M = 481 D = 203 
I = 685 M = 482 D = 203 
I = 686 M = 483 D = 203 
I = 687 M = 484 D = 203 
I = 688 M = 485 D = 203 
I = 689 M = 486 D = 203 
I = 690 M = 487 D = 203 
I = 691 M = 488 D = 203 
I = 692 M = 489 D = 203 
I = 693 M = 489 D = 204 
I = 694 M = 490 D = 204 
I = 695 M = 491 D = 204 
I = 696 M = 492 D = 204 
I = 697 M = 492 D = 205 
I = 698 M = 493 D = 205 
I = 699 M = 494 D = 205 
I = 700 M = 495 D = 205 
I = 701 M = 496 D = 205 
I = 702 M = 497 D = 205 
I = 703 M = 498 D = 205 
I = 704 M = 499 D = 205 
I = 705 M = 500 D = 205 
I = 706 M = 500 D = 206 
I = 707 M = 501 D = 206 
I = 708 M = 502 D = 206 
I = 709 M = 503 D = 206 
I = 710 M = 504 D = 206 
I = 711 M = 505 D = 206 
I = 712 M = 506 D = 206 
I = 713 M = 506 D = 207 
I = 714 M = 507 D = 207 
I = 715 M = 508 D = 207 
I = 716 M = 508 D = 208 
I = 717 M = 509 D = 208 
I = 718 M = 510 D = 208 
I = 719 M = 511 D = 208 


I = 1007 M = 725 D = 282 
I = 1008 M = 726 D = 282 
I = 1009 M = 727 D = 282 
I = 1010 M = 728 D = 282 
I = 1011 M = 728 D = 283 
I = 1012 M = 729 D = 283 
I = 1013 M = 730 D = 283 
I = 1014 M = 731 D = 283 
I = 1015 M = 732 D = 283 
I = 1016 M = 733 D = 283 
I = 1017 M = 733 D = 284 
I = 1018 M = 734 D = 284 
I = 1019 M = 735 D = 284 
I = 1020 M = 735 D = 285 
I = 1021 M = 736 D = 285 
I = 1022 M = 737 D = 285 
I = 1023 M = 738 D = 285 
I = 1024 M = 739 D = 285 
I = 1025 M = 740 D = 285 
I = 1026 M = 741 D = 285 
I = 1027 M = 742 D = 285 
I = 1028 M = 743 D = 285 
I = 1029 M = 743 D = 286 
I = 1030 M = 744 D = 286 
I = 1031 M = 745 D = 286 
I = 1032 M = 746 D = 286 
I = 1033 M = 747 D = 286 
I = 1034 M = 748 D = 286 
I = 1035 M = 748 D = 287 
I = 1036 M = 748 D = 288 
I = 1037 M = 749 D = 288 
I = 1038 M = 750 D = 288 
I = 1039 M = 751 D = 288 
I = 1040 M = 751 D = 289 
I = 1041 M = 751 D = 290 
I = 1042 M = 751 D = 291 
I = 1043 M = 752 D = 291 
I = 1044 M = 753 D = 291 
I = 1045 M =

I = 1322 M = 966 D = 356 
I = 1323 M = 967 D = 356 
I = 1324 M = 968 D = 356 
I = 1325 M = 969 D = 356 
I = 1326 M = 970 D = 356 
I = 1327 M = 971 D = 356 
I = 1328 M = 971 D = 357 
I = 1329 M = 972 D = 357 
I = 1330 M = 973 D = 357 
I = 1331 M = 973 D = 358 
I = 1332 M = 974 D = 358 
I = 1333 M = 975 D = 358 
I = 1334 M = 976 D = 358 
I = 1335 M = 977 D = 358 
I = 1336 M = 978 D = 358 
I = 1337 M = 978 D = 359 
I = 1338 M = 979 D = 359 
I = 1339 M = 980 D = 359 
I = 1340 M = 981 D = 359 
I = 1341 M = 982 D = 359 
I = 1342 M = 983 D = 359 
I = 1343 M = 983 D = 360 
I = 1344 M = 983 D = 361 
I = 1345 M = 984 D = 361 
I = 1346 M = 985 D = 361 
I = 1347 M = 986 D = 361 
I = 1348 M = 986 D = 362 
I = 1349 M = 987 D = 362 
I = 1350 M = 987 D = 363 
I = 1351 M = 988 D = 363 
I = 1352 M = 989 D = 363 
I = 1353 M = 990 D = 363 
I = 1354 M = 991 D = 363 
I = 1355 M = 992 D = 363 
I = 1356 M = 993 D = 363 
I = 1357 M = 994 D = 363 
I = 1358 M = 994 D = 364 
I = 1359 M = 995 D = 364 
I = 1360 M =

I = 1627 M = 1198 D = 429 
I = 1628 M = 1199 D = 429 
I = 1629 M = 1200 D = 429 
I = 1630 M = 1201 D = 429 
I = 1631 M = 1202 D = 429 
I = 1632 M = 1203 D = 429 
I = 1633 M = 1203 D = 430 
I = 1634 M = 1204 D = 430 
I = 1635 M = 1205 D = 430 
I = 1636 M = 1206 D = 430 
I = 1637 M = 1207 D = 430 
I = 1638 M = 1207 D = 431 
I = 1639 M = 1208 D = 431 
I = 1640 M = 1209 D = 431 
I = 1641 M = 1210 D = 431 
I = 1642 M = 1211 D = 431 
I = 1643 M = 1212 D = 431 
I = 1644 M = 1213 D = 431 
I = 1645 M = 1214 D = 431 
I = 1646 M = 1215 D = 431 
I = 1647 M = 1216 D = 431 
I = 1648 M = 1217 D = 431 
I = 1649 M = 1218 D = 431 
I = 1650 M = 1219 D = 431 
I = 1651 M = 1220 D = 431 
I = 1652 M = 1220 D = 432 
I = 1653 M = 1221 D = 432 
I = 1654 M = 1221 D = 433 
I = 1655 M = 1222 D = 433 
I = 1656 M = 1223 D = 433 
I = 1657 M = 1224 D = 433 
I = 1658 M = 1225 D = 433 
I = 1659 M = 1226 D = 433 
I = 1660 M = 1226 D = 434 
I = 1661 M = 1227 D = 434 
I = 1662 M = 1228 D = 434 
I = 1663 M = 1229 D = 434 
I

I = 1930 M = 1427 D = 503 
I = 1931 M = 1428 D = 503 
I = 1932 M = 1429 D = 503 
I = 1933 M = 1430 D = 503 
I = 1934 M = 1430 D = 504 
I = 1935 M = 1431 D = 504 
I = 1936 M = 1432 D = 504 
I = 1937 M = 1432 D = 505 
I = 1938 M = 1433 D = 505 
I = 1939 M = 1434 D = 505 
I = 1940 M = 1435 D = 505 
I = 1941 M = 1436 D = 505 
I = 1942 M = 1436 D = 506 
I = 1943 M = 1437 D = 506 
I = 1944 M = 1438 D = 506 
I = 1945 M = 1439 D = 506 
I = 1946 M = 1439 D = 507 
I = 1947 M = 1440 D = 507 
I = 1948 M = 1441 D = 507 
I = 1949 M = 1442 D = 507 
I = 1950 M = 1443 D = 507 
I = 1951 M = 1444 D = 507 
I = 1952 M = 1445 D = 507 
I = 1953 M = 1446 D = 507 
I = 1954 M = 1446 D = 508 
I = 1955 M = 1446 D = 509 
I = 1956 M = 1446 D = 510 
I = 1957 M = 1447 D = 510 
I = 1958 M = 1447 D = 511 
I = 1959 M = 1447 D = 512 
I = 1960 M = 1448 D = 512 
I = 1961 M = 1448 D = 513 
I = 1962 M = 1449 D = 513 
I = 1963 M = 1450 D = 513 
I = 1964 M = 1451 D = 513 
I = 1965 M = 1452 D = 513 
I = 1966 M = 1452 D = 514 
I

I = 2233 M = 1651 D = 582 
I = 2234 M = 1652 D = 582 
I = 2235 M = 1652 D = 583 
I = 2236 M = 1652 D = 584 
I = 2237 M = 1653 D = 584 
I = 2238 M = 1654 D = 584 
I = 2239 M = 1654 D = 585 
I = 2240 M = 1655 D = 585 
I = 2241 M = 1656 D = 585 
I = 2242 M = 1657 D = 585 
I = 2243 M = 1657 D = 586 
I = 2244 M = 1657 D = 587 
I = 2245 M = 1657 D = 588 
I = 2246 M = 1657 D = 589 
I = 2247 M = 1658 D = 589 
I = 2248 M = 1659 D = 589 
I = 2249 M = 1660 D = 589 
I = 2250 M = 1661 D = 589 
I = 2251 M = 1661 D = 590 
I = 2252 M = 1662 D = 590 
I = 2253 M = 1663 D = 590 
I = 2254 M = 1664 D = 590 
I = 2255 M = 1664 D = 591 
I = 2256 M = 1664 D = 592 
I = 2257 M = 1665 D = 592 
I = 2258 M = 1666 D = 592 
I = 2259 M = 1667 D = 592 
I = 2260 M = 1667 D = 593 
I = 2261 M = 1668 D = 593 
I = 2262 M = 1668 D = 594 
I = 2263 M = 1669 D = 594 
I = 2264 M = 1669 D = 595 
I = 2265 M = 1670 D = 595 
I = 2266 M = 1671 D = 595 
I = 2267 M = 1672 D = 595 
I = 2268 M = 1673 D = 595 
I = 2269 M = 1673 D = 596 
I

In [14]:
# evaluate and print the test accuracy
# get index of predicted smile detection for each image in test set
smile_prediction = [np.argmax(newModel.predict(np.expand_dims(test_data, axis=0))) for test_data in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(smile_prediction)==np.argmax(test_targets, axis=1))/len(smile_prediction)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 74.2800%



Here, we see that an accuracy of 74.28% is achieved with this model.